In [1]:
!pip install skforecast
!pip install dask[dataframe]
!pip install skforecast
!pip install dask[dataframe]
!pip install lightgbm
!pip install matplotlib
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import warnings
import csv

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

warnings.filterwarnings('once')

C:\Users\juanbaroffi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
steps = [5,10,15,20,25,30]
lags = [2,5,7,10]

In [4]:
data = pd.read_csv('../FitBit API Database/test_train_data_api_merged.csv')
print("Archivo : test_train_data_api_merged.csv")
print(data)

Archivo : test_train_data.csv
                Id                 Time   HeartRate  Intensity  Calories
0       2022484408  2016-04-12 07:21:00  101.600000          1   3.32064
1       2022484408  2016-04-12 07:22:00   87.888889          1   3.94326
2       2022484408  2016-04-12 07:23:00   58.000000          0   1.34901
3       2022484408  2016-04-12 07:24:00   58.000000          0   1.03770
4       2022484408  2016-04-12 07:25:00   56.777778          0   1.03770
...            ...                  ...         ...        ...       ...
333141  8877689391  2016-05-12 13:55:00   60.666667          0   1.33353
333142  8877689391  2016-05-12 13:56:00   61.875000          0   1.33353
333143  8877689391  2016-05-12 13:57:00   58.142857          0   1.33353
333144  8877689391  2016-05-12 13:58:00   61.200000          0   1.33353
333145  8877689391  2016-05-12 13:59:00   58.000000          0   1.33353

[333146 rows x 5 columns]


Archivo : test_train_data_api_merged.csv
            Id          

In [5]:
modelos = [
    ("LightGBM", lgb.LGBMRegressor(objective='regression', random_state=123)),
    ("XGBoost", XGBRegressor(random_state=123)),
    ("KNN", KNeighborsRegressor(n_neighbors=5)),
    ("RandomForest", RandomForestRegressor(n_estimators=100, max_depth=None, random_state=123)),
    ("MLP",MLPRegressor(random_state=123, max_iter=2000))
]


In [6]:
resultados = {}

for nombre_modelo, modelo in modelos:
    for step in steps:  
        datos_train = data.iloc[:-step]  
        datos_test  = data.iloc[-step:] 
        datos_train = datos_train.bfill();
        datos_test = datos_test.bfill();

        for lag in lags:
            forecaster = ForecasterAutoreg(
                regressor=modelo,
                lags=lag
            )
            forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories', 'Steps', 'Distance']])
            predicciones = forecaster.predict(steps=step, exog=datos_test[['Calories', 'Steps', 'Distance']])
            error_mse = mean_squared_error(y_true=datos_test['HeartRate'], y_pred=predicciones)
            resultados[(nombre_modelo, step, lag)] = error_mse
            



nombre_archivo = "../resultados/resultados_modelos.csv"

# Abrir el archivo CSV en modo de escritura
with open(nombre_archivo, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Escribir el encabezado del archivo CSV
    writer.writerow(['Modelo', 'Steps', 'Lags', 'MSE'])

    # Escribir los resultados en el archivo CSV
    for nombre_modelo, modelo in modelos:
        for step in steps:
            for lag in lags:
                error_mse = resultados[(nombre_modelo, step, lag)]
                # Escribir una fila en el archivo CSV
                writer.writerow([nombre_modelo, step, lag, error_mse])

print(f"Los resultados se han guardado en el archivo '{nombre_archivo}'.")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 756
[LightGBM] [Info] Number of data points in the train set: 103102, number of used features: 5
[LightGBM] [Info] Start training from score 65.475316
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1128
[LightGBM] [Info] Number of data points in the train set: 103099, number of used features: 8
[LightGBM] [Info] Start training from score 65.475494
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001979 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enou

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import lightgbm as lgb
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import csv

# Cargar datos
data = pd.read_csv('../app/app/DataAPI/BSGSML/test_train_data_api_merged.csv')

# Convertir la columna 'Time' a datetime
data['Time'] = pd.to_datetime(data['Time'])

# Establecer 'Time' como índice
data.set_index('Time', inplace=True)

print("Archivo : test_train_data_api_merged.csv")
print(data)

modelos = [
    ("LightGBM", lgb.LGBMRegressor(objective='regression', random_state=123))
]

steps = [10]  # Reemplaza con los valores que estás utilizando
lags = [15]   # Reemplaza con los valores que estás utilizando

resultados = {}
predicciones_dict = {}

def graficar_predicciones(datos_test, predicciones_dict, future_index):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Graficar los datos reales
    ax.plot(datos_test.index, datos_test['HeartRate'], label='Datos Reales', color='black', linewidth=2)

    # Graficar las predicciones de cada modelo
    for (nombre_modelo, step, lag), predicciones in predicciones_dict.items():
        # Alinear las predicciones con el índice de future_index
        predicciones_aligned = pd.Series(predicciones.values, index=future_index)
        ax.plot(predicciones_aligned.index, predicciones_aligned, label=f'{nombre_modelo} (step={step}, lag={lag})')

    # Configurar etiquetas y leyenda
    ax.set_xlabel('Tiempo')
    ax.set_ylabel('HeartRate')
    ax.legend()
    plt.xticks(rotation=45)
    
    # Configurar formato de fecha en el eje x
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))
    fig.autofmt_xdate()
    
    plt.show()

# Reindexar los datos para asegurarse de que el índice tenga una frecuencia
data = data.asfreq('T')

# Generar un índice de tiempo futuro a partir del último punto de datos
last_time = data.index[-1]
future_index = pd.date_range(start=last_time, periods=steps[0]+1, freq='T')[1:]

# Generar datos ficticios para las variables exógenas en el futuro
future_exog = pd.DataFrame({
    'Calories': np.random.uniform(low=0, high=5, size=len(future_index)),
    'Steps': np.random.randint(low=0, high=100, size=len(future_index)),
    'Distance': np.random.uniform(low=0, high=1, size=len(future_index))
}, index=future_index)

for nombre_modelo, modelo in modelos:
    for step in steps:  
        datos_train = data.iloc[:]  
        datos_train = datos_train.bfill()

        for lag in lags:
            forecaster = ForecasterAutoreg(
                regressor=modelo,
                lags=lag
            )
            forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories', 'Steps', 'Distance']])
            predicciones = forecaster.predict(steps=step, exog=future_exog)
            predicciones_dict[(nombre_modelo, step, lag)] = predicciones
            graficar_predicciones(datos_train, predicciones_dict, future_index)